In [ ]:
import os
import yaml
from scipy.special import expit
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from regionmask import mask_geopandas
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean.cm as cmo

plt.rcParams["font.family"] = ["serif", "sans-serif", "monospace"][2]

bd = os.path.join("/hn01-home", "spet5107") # depends on device
os.chdir(bd)

# settings
k = 1
wd = ["/hazGAN2/projects/bayofbengal_era5", "hazGAN2/projects/poweruk_winter"][k]
std_string = ["gumbel", "standardised"][k]
v1, v2, v3 = [("ws", "tp", "msl"), ("u10_gust", "v10_gust", "r30")][k]

# choose φ-level
generated_suffix = ["", "_trunc04", "_trunc04_old"][0]

with open(os.path.join(wd, "config.yaml"), "r") as stream:
    config = yaml.safe_load(stream)

regions = ["East Midlands", "West Midlands", "South West England", "South Wales"]
region_of_interest = regions[0]

season = config["sfunc"]
local_crs = config["local_crs"]

print("Using {} domain.".format(config["domain"]))

In [ ]:
def set_size(width, fraction=1):
    """ Set aesthetic figure dimensions to avoid scaling in latex.
    https://tobiasraabe.github.io/post/matplotlib-for-publications/

    Parameters
    ----------
    width: float
            Width in pts
    fraction: float
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    golden_ratio = (5 ** 0.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    return fig_width_in, fig_height_in

def laplace(uniform, mu=0, b=1):
    """uniform -> Laplace(mu, b) (quantile function)"""
    maxval = np.max(uniform)
    if maxval == 1:
        warn("Values == 1 found, scaling by 1e-6")
        uniform *= 1 - 1e-6
    if maxval > 1:
        raise ValueError(f"Some uniform > 1 ({maxval})")
    
    return np.where(
        uniform <= 0.5, 
        mu + b * np.log(2 * uniform),
        mu - b * np.log(2 - 2 * uniform)
        )


def inv_laplace(x, mu=0, b=1):
    """Laplace(mu, b) -> uniform (CDF function)."""
    return np.where(
        x <= mu,
        0.5 * np.exp((x - mu) / b),
        1 - 0.5 * np.exp(-(x - mu) / b)
    )


def gumbel(uniform):
    """uniform -> Gumbel(0, 1)"""
    maxval = np.max(uniform)
    if maxval == 1:
        print("Values == 1 found, scaling by 1e-6")
        uniform *= 1 - 1e-6
    if maxval > 1:
        raise ValueError(f"Some uniform > 1 ({maxval})")
    return -np.log(-np.log(uniform))

In [ ]:
rp_max = 10_000

print(f"Laplace max for {rp_max} return period: {laplace(1 - 1/rp_max)}")
print(f"Gumbel max for {rp_max} return period: {gumbel(1 - 1/rp_max)}")
print(f"Laplace min for {rp_max} return period: {laplace(1/rp_max)}")
print(f"Gumbel min for {rp_max} return period: {gumbel(1/rp_max)}")

print(laplace(0.999) - laplace(0.99))
print(gumbel(0.999) - gumbel(0.99))
print("\nSo laplace stretches the tails very slightly less.")

In [ ]:
train = xr.open_dataset(os.path.join(wd, "results", "training", "data.nc"))
train["standardised"] = (["time", "lat", "lon", "field"], laplace(train["uniform"]).data)

print(train.sizes)

gener = xr.open_dataset(os.path.join(wd, "results", f"generated{generated_suffix}", "netcdf", "data.nc"), engine="netcdf4")
gener.load() # do the expensive server load once

print(gener.sizes)
train["params"].isel(field=0, param=0).plot()

In [ ]:
from typing import Tuple

def load_events(df) -> Tuple[gpd.GeoDataFrame, pd.Series, float]:
    """Load events data and metadata, return GeoDataFrame and event times."""
    df.columns = [col.replace(".", "_") for col in df.columns]
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]), crs="EPSG:4326")
    gdf["lat"] = gdf.geometry.y
    gdf["lon"] = gdf.geometry.x
    gdf = gdf.sort_values(["event", "lat", "lon"]).reset_index(drop=True)
    return gdf


def create_parameters(
        gdf:gpd.GeoDataFrame, h:int, w:int, fields:list
        ) -> np.ndarray:
    """Create hxwx6xk parameter array for distribution parameters."""
    params = np.full((h, w, 6, len(fields)), np.nan, dtype=np.float32)
    
    prefixes = ["thresh_", "scale_", "shape_"]
    suffixes = ["", "_lower", "_upper"]
    param_map = {0: 3, 1: 0, 2: 0}  # suffix index to parameter offset
    
    for k, field in enumerate(fields):
        for i, suffix in enumerate(suffixes):
            for j, prefix in enumerate(prefixes):
                col = f"{prefix}{field}{suffix}"
                if col in gdf.columns:
                    values = gdf[[col, "lat", "lon"]].groupby(["lat", "lon"]).mean()[col].values
                    params[:, :, j + param_map[i], k] = values.reshape([h, w])
    
    return params

fits = pd.read_parquet(os.path.join(wd, "results", "processing", "fitted.parquet"))
fits.head()
gdf = load_events(fits)

In [ ]:
gdf.head()

#### Why are all the parameters NA?

Re-run with 
```bash
snakemake --profile profiles/slurm projects/poweruk_winter/results/processing/fitted.parquet -n
```

Then run

```python
fits = pd.read_parquet(os.path.join(wd, "results", "processing", "fitted.parquet"))
fits.head()
```


In [ ]:
fits = pd.read_parquet(os.path.join(wd, "results", "processing", "fitted.parquet"))
fits.head()

In [ ]:
hist_kws = {"edgecolor": 'dimgrey', "linewidth": 0.5, "bins": 50, "density": True}

fig, axs = plt.subplots(1, 3, figsize=(10, 3))

i = 0

domain_train = ["standardised", "uniform", "anomaly"][i] # check the other one
domain_gener = [std_string, "uniform", "anomaly"][i]

q = 0.0

ax = axs[0]
y0 = train.sel(field=v1)[domain_train].max(dim=["lat", "lon"]).values.ravel()
y1 = gener.sel(field=v1)[domain_gener].max(dim=["lat", "lon"]).values.ravel()
dy = (gener.sel(field=v1)[domain_gener].max(dim=["time"]) - train.sel(field=v1)[domain_train].max(dim=["time"])).mean().item()
dy_max = (gener.sel(field=v1)[domain_gener].max(dim=["time"]) - train.sel(field=v1)[domain_train].max(dim=["time"])).max().item()
threshold = np.quantile(y0, q)
y0_tail = y0[y0 > threshold]
y1_tail = y1[y1 > threshold]
ax.hist(y1_tail, color="lightblue", label="Generated", **hist_kws);
ax.hist(y0_tail, color="white", alpha=0.6, label="Training", **hist_kws);
ax.set_xlabel(r"Max $\lambda u_{10}$ anomaly (ms$^{-1})$")
print("{} train max: {}, generated max: {}, avg. diff: {}, max. diff: {}".format(v1, y0.max(), y1.max(), dy, dy_max))

ax = axs[1]
y0 = train.sel(field=v2)[domain_train].max(dim=["lat", "lon"]).values.ravel()
y1 = gener.sel(field=v2)[domain_gener].max(dim=["lat", "lon"]).values.ravel()
dy = (gener.sel(field=v2)[domain_gener].max(dim=["time"]) - train.sel(field=v2)[domain_train].max(dim=["time"])).mean().item()
dy_max = (gener.sel(field=v2)[domain_gener].max(dim=["time"]) - train.sel(field=v2)[domain_train].max(dim=["time"])).max().item()
threshold = np.quantile(y0, q)
y0_tail = y0[y0 > threshold]
y1_tail = y1[y1 > threshold]
ax.hist(y1_tail, color="lightblue", label="Generated", **hist_kws);
ax.hist(y0_tail, color="white", alpha=0.6, label="Training", **hist_kws);
ax.set_xlabel(r"Max $\lambda v_{10}$ anomaly [ms$^{-1}$]")
print("{} train max: {}, generated max: {}, avg. diff: {}, max. diff: {}".format(v2, y0.max(), y1.max(), dy, dy_max))

ax = axs[2]
y0 = train.sel(field=v3)[domain_train].mean(dim=["lat", "lon"]).values.ravel()
y1 = gener.sel(field=v3)[domain_gener].mean(dim=["lat", "lon"]).values.ravel()
dy = (gener.sel(field=v3)[domain_gener].max(dim=["time"]) - train.sel(field=v3)[domain_train].max(dim=["time"])).mean().item()
dy_max = (gener.sel(field=v3)[domain_gener].max(dim=["time"]) - train.sel(field=v3)[domain_train].max(dim=["time"])).max().item()
threshold = np.quantile(y0, q)
y0_tail = y0[y0 > threshold]
y1_tail = y1[y1 > threshold]
ax.hist(y1_tail, color="lightblue", label="Generated", **hist_kws);
ax.hist(y0_tail, color="white", alpha=0.6, label="Training", **hist_kws);
ax.set_xlabel(r"Mean $r^{30}$ anomaly [m]")
ax.legend(loc="center right")
print("{} train max: {}, generated max: {}, avg. diff: {}, max. diff: {}".format(v3, y0.max(), y1.max(), dy, dy_max))

for ax in axs:
    ax.spines[['right', 'top']].set_visible(False)
    ax.set_ylabel("Count")
    ax.label_outer()

plt.tight_layout()
fig.suptitle(f"Overall")

fig.savefig(os.path.join(
    wd, "results", "figures", "tail_hists.png"), dpi=300, transparent=True, bbox_inches="tight")

In [ ]:
# temp, view train only
def format_p(p):
    if isinstance(p, list):
        return {"lon": p[0], "lat": p[1]}
    else:
        return p
    
p = [format_p(p) for p in config["points_of_interest"].values()][1]
hist_kws = {"edgecolor": 'dimgrey', "linewidth": 0.5, "bins": 50, "density": True}

fig, axs = plt.subplots(1, 3, figsize=(10, 3))

i = 2

domain_train = ["standardised", "uniform", "anomaly"][i] # check the other one
domain_gener = [std_string, "uniform", "anomaly"][i]

q = 0.0

ax = axs[0]
y0 = train.sel(field=v1)[domain_train].sel(**p, method="nearest").values.ravel()
y1 = gener.sel(field=v1)[domain_gener].sel(**p, method="nearest").values.ravel()
threshold = np.quantile(y0, q)
y0_tail = y0[y0 > threshold]
y1_tail = y1[y1 > threshold]
ax.hist(y1_tail, color="lightblue", label="Generated", **hist_kws);
ax.hist(y0_tail, color="white", alpha=0.6, label="Training", **hist_kws);
ax.set_xlabel(r"Max $\lambda u_{10}$ anomaly (ms$^{-1})$")
print("{} train max: {}, generated max: {}, diff: ".format(v1, y0.max(), y1.max()), y1.max() - y0.max())

ax = axs[1]
y0 = train.sel(field=v2)[domain_train].sel(**p, method="nearest").values.ravel()
y1 = gener.sel(field=v2)[domain_gener].sel(**p, method="nearest").values.ravel()
threshold = np.quantile(y0, q)
y0_tail = y0[y0 > threshold]
y1_tail = y1[y1 > threshold]
ax.hist(y1_tail, color="lightblue", label="Generated", **hist_kws);
ax.hist(y0_tail, color="white", alpha=0.6, label="Training", **hist_kws);
ax.set_xlabel(r"Max $\lambda v_{10}$ anomaly [ms$^{-1}$]")
print("{} train max: {}, generated max: {}, diff: ".format(v2, y0.max(), y1.max()), y1.max() - y0.max())

ax = axs[2]
y0 = train.sel(field=v3)[domain_train].sel(**p, method="nearest").values.ravel()
y1 = gener.sel(field=v3)[domain_gener].sel(**p, method="nearest").values.ravel()
threshold = np.quantile(y0, q)
y0_tail = y0[y0 > threshold]
y1_tail = y1[y1 > threshold]
ax.hist(y1_tail, color="lightblue", label="Generated", **hist_kws);
ax.hist(y0_tail, color="white", alpha=0.6, label="Training", **hist_kws);
ax.set_xlabel(r"Mean $r^{30}$ anomaly [m]")
ax.legend(loc="center right")
ax.set_title(f"At point {p['lat']:.2f}N, {p['lon']:.2f}E")
print("{} train max: {}, generated max: {}, diff: ".format(v3, y0.max(), y1.max()), y1.max() - y0.max())

for ax in axs:
    ax.spines[['right', 'top']].set_visible(False)
    ax.set_ylabel("Count")
    ax.label_outer()

plt.tight_layout()
# fig.suptitle(f"Thresholding tail above {q*100:.0f}th quantile ({domain_train})")

fig.savefig(os.path.join(
    wd, "results", "figures", "tail_hist_marginal.png"), dpi=300, transparent=True, bbox_inches="tight")

In [ ]:
np.array([field['two_tailed'] for field in config["fields"].values()]).any()

In [ ]:
rp_max = 10_000
p_max = 1 - 1/rp_max

print(f"Using p_max = {p_max} for {rp_max} return period")


os.chdir(os.path.join(bd, "hazGAN2", "workflow"))
from importlib import reload
import src.python.statistics as stats
reload(stats)
os.chdir(bd)

# do the same as in process_generated.py
train_x = train["anomaly"].values
theta  = train["params"].values

# transform images to original scale using invPIT
distns = [field['distn'] for field in config["fields"].values()]
two_tailed =[field['two_tailed'] for field in config["fields"].values()]

if False:
    # option to force one-tailed
    two_tailed = [False, False, False]
    theta = theta[..., :3, :]  # drop lower/upper if one-tailed

test_u = p_max * np.ones_like(train_x)
upper_bounds = stats.invPIT(test_u, train_x, theta=theta, distns=distns, two_tailed=two_tailed)

# Heatmaps
fig, axs = plt.subplots(1, 3, figsize=(9, 2))
train_maxima = train.isel(field=0)["uniform"].max(dim=["time"]).values
im = axs[0].imshow(train_maxima[::-1, :])
plt.colorbar(im, ax=axs[0])
print(f"Train maxima uniform: {train_maxima.max()}")
train_maxima = train.isel(field=0)["anomaly"].max(dim=["time"]).values
im = axs[1].imshow(upper_bounds[0, ::-1, :, 0])
plt.colorbar(im, ax=axs[1])
im = axs[2].imshow(train_maxima[::-1, :])
plt.colorbar(im, ax=axs[2])

# Scatter plot
fig, ax = plt.subplots(figsize=(2, 2))
ax.scatter(train_maxima.ravel(), upper_bounds[0, :, :, 0].ravel(), s=1)

In [ ]:
gener2 = gener.copy()
new_gener = stats.invPIT(gener["uniform"].values, train_x, theta=theta, distns=distns, two_tailed=two_tailed)
gener2["anomaly"] = (["time", "lat", "lon", "field"], new_gener)

In [ ]:
# temp, view train only
p0, p1 = [format_p(p) for p in config["points_of_interest"].values()][1:3]
n0, n1 = [name for name in config["points_of_interest"].keys()][1:3]

hist_kws = {"edgecolor": 'dimgrey', "linewidth": 0.5, "bins": 50, "density": True}

fig, axs = plt.subplots(1, 2, figsize=(6, 2))


j = 1
k = v3

domain_train = ["standardised", "uniform", "anomaly"][j] # check the other one
domain_gener = [std_string, "uniform", "anomaly"][j]
print(f"Plotting in {domain_train} and {domain_gener} domains.")

ax = axs[0]
y0 = train.sel(field=k)[domain_train].sel(**p0, method="nearest").values.ravel()
y1 = train.sel(field=k)[domain_train].sel(**p1, method="nearest").values.ravel()
ax.scatter(y0, y1, color='k', s=5)
y0_min, y0_max = y0.min(), y0.max()
y1_min, y1_max = y1.min(), y1.max()
ax.set_title(f"Train\n({domain_train})")

ax = axs[1]
y0 = gener.sel(field=k)[domain_gener].sel(**p0, method="nearest").values.ravel()
y1 = gener.sel(field=k)[domain_gener].sel(**p1, method="nearest").values.ravel()
ax.scatter(y0, y1, color='k', s=5)
ax.set_title(f"Generated\n({domain_gener})")

for ax in axs:
    ax.axhline(y1_min, color="red", linestyle="dashed")
    ax.axhline(y1_max, color="red", linestyle="dashed")
    ax.axvline(y0_min, color="red", linestyle="dashed")
    ax.axvline(y0_max, color="red", linestyle="dashed")
    ax.set_xlabel(n0.title())
    ax.set_ylabel(n1.title())
    ax.label_outer();

    if domain_train == "uniform":
        # ax.set_ylim(0, 1)
        # ax.set_xlim(0, 1)
        ax.set_xscale("logit")
        ax.set_yscale("logit")
    else:
        pass
        ax.set_ylim(min(y1_min, y0.min()), max(y1_max, y1.max()))

outpath = os.path.join(wd, "results", "figures", f"scatter_{k}{generated_suffix}.png")
fig.savefig(outpath, dpi=300, transparent=True, bbox_inches="tight")

print("Saved as {}".format(outpath))